<a href="https://colab.research.google.com/github/sherirosalia/PPP_Covid_Stimulus/blob/master/random_forest/ML_150.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Over 150k random forest analysis ** 

Importing the data csv
 

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
import os, sys 
import io
import ee


In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1000)

In [ ]:
#mounting the google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# command shows file path
# ! ls

In [ ]:
from google.colab import files

In [ ]:
# filepath
path = "/content/drive/My Drive/data/PPP Data 150k plus.csv"


In [ ]:
# Read column names from file
cols = list(pd.read_csv(path, nrows =1))
print(cols)

In [ ]:
def conv(val):
    if val == np.nan:
        return 0 # or whatever else you want to represent your NaN with
    return val

In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
# allowing an index column
ndf = pd.read_csv(path,  usecols =[i for i in cols if i != 'NonProfit'], dtype={"Zip":str, "NAICSCode": str,  }, parse_dates=[12],)
# df = pd.read_csv(path, error_bad_lines=False)
ndf.sample(3)

In [ ]:
# read the csv, create dataframe
# basic option
# df = pd.read_csv(path,)


In [ ]:
# check
# df.sample()

Checking for nulls and changing types.

In [ ]:
#will need to change datatypes
ndf.info()

In [ ]:
#drop the non-profit column
# df=df.drop(['NonProfit'],axis=1)
# df.shape

In [ ]:
#check for nulls
print(ndf.isna().sum())


In [ ]:
# drop if only a few null values 
ndf.dropna(inplace=True)
# df.dtypes

In [ ]:
#check for nulls
print(ndf.isna().sum())

In [ ]:
#shape
ndf.shape

In [ ]:
ndf.sample()




In [ ]:
ndf.to_csv('no_nulls.csv', index=0)
!cp no_nulls.csv "drive/My Drive/"

In [ ]:
no_nulls= "/content/no_nulls.csv"


In [ ]:
# df = pd.read_csv(csv_path, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, }, parse_dates=[10] )


In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
# allowing an index column

df = pd.read_csv(no_nulls, dtype={'NAICSCode': str, "JobsRetained": int, "Zip":str,  }, parse_dates=[12])
# df = pd.read_csv(path, error_bad_lines=False)
df.sample()

In [ ]:
df.info()

Preserving next few cells, but do not need with this import

In [ ]:
# I don't need the following cells for this particular import
# naics saved should not be a float. Need to convert to string before integer
# df['NAICS Code']=df['NAICS Code'].astype(str)

In [ ]:
#strip the decimal and zero
# df['NAICS Code'] = df['NAICS Code'].str.rstrip('.0')

In [ ]:
#take the first two letters of the NAICS code column and create category column. 
#category column can be matched to named values. The longer code was impractical
# df['NAICS Category'] = df['NAICS Code'].str[:2]
# df.info()

In [ ]:
# df.sample(7)

In [ ]:
#somewhat forcing the date conversion because there must be some values that are not in date format 
# df['Date'] =  pd.to_datetime(df['Date'], errors='coerce')

In [ ]:
#converting to date worked
# df.info()

In [ ]:
# May need for machine learning prep: 
# df['Jobs Saved'] = pd.to_numeric(df['Jobs Saved'], downcast='float')
# df['NAICS Category'] = pd.to_numeric(df['NAICS Category'], downcast='float')

In [ ]:
# strip off the decimal point if there is one
# df['Jobs Saved'] = df['Jobs Saved'].str.rstrip('.0')
# df['Jobs Saved'] = df['Jobs Saved'].astype(int)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:

df.sample()

In [ ]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()


In [ ]:
# rather than state a copy of df, and to avoid setcopy warnings, create a new dataframe
data=pd.DataFrame()

In [ ]:
# # label encoder
for x in ['Gender', 'RaceEthnicity', 'Veteran', 'LoanRange', 'Lender','State','DateApproved', 'BusinessType']:
  label_encoder.fit(df[x])
  data[x] = label_encoder.transform(df[x])
#data['JobsRetained'] = df['JobsRetained']
data['NAICS Category'] = df['NAICSCode']
# data['Loan Amount'] = df['Loan Amount']

In [ ]:
data.shape


In [ ]:
data.sample()

In [ ]:
# hot encode
# one = OneHotEncoder(sparse=False)

In [ ]:
# one.fit_transform(data)

In [ ]:
# data['Jobs Saved'].shape


In [ ]:
#split loan amounts off and drop from dataframe for ML
target=data['LoanRange'] 
target.shape


In [ ]:
data=data.drop(['LoanRange'],axis=1)
data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

**Results:**

Score with: Gender | RaceEthnicity | Veteran |   JobsRetained | NAICS | Lender = 0.61278778

Score with: Gender | RaceEthnicity | Veteran |  BusinessType | JobsRetained | NAICS | DateApproved | State | LenderCategory = -.65

Score with: Gender | RaceEthnicity | Veteran |   NAICS | Lender -0.006 ---!

Score with: Gender | RaceEthnicity | Veteran |  BusinessType | JobsRetained | NAICS | Lender = 0.19463591101903663

ML suggests that Jobs Saved and Business Type influence/correlate with the amount of money loaned by the bank. Does not necessarily suggest whether or not one was approved or not.





In [ ]:
#score of forest
print(rf.score(X_test, y_test))



In [ ]:
#zip names and feature importance scores
f_imp=rf.feature_importances_
f_name=data.columns
features=zip(f_name, f_imp)
# print(features)
# features into a datafame
f_df=pd.DataFrame(list(features))
print(f_df)


In [ ]:
# X_train.columns
print(rf.feature_importances_)

In [ ]:
data.to_csv('large_ml.csv', index=0)
!cp people_ml.csv "drive/My Drive/"